## Assignment 7.1

## a.

In [3]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil
import pandas as pd
import pygeohash
import s3fs
endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
    return records
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    return flat_record
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])
df = create_flattened_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [14]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_longitude,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B


In [33]:
# df['kv_key'] = df['key'].str[0]

In [111]:
# df.head()

In [36]:
# df.to_parquet("./results/kv/",partition_cols=['kv_key'])

In [38]:
partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )

In [109]:
def def_part(z):
    for x in partitions:
        if z in x:
            if x[0] == x[1]:
                return(x[0])
            else:
                return(x[0]+"-"+x[1])

In [110]:
print(def_part('K'))

K-L


In [114]:
df = df.drop(['kv_key'], axis = 1)

In [118]:
df['kv_key'] = df['key'].str[0].apply(def_part)

In [119]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D


In [120]:
df.to_parquet("./results/kv/",partition_cols=['kv_key'])

## b.

In [121]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [126]:
df['hashed'] = df['key'].apply(hash_key)

In [131]:
df['hash_key'] = df['hashed'].str[0]

In [132]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1


In [133]:
df.to_parquet("./results/hash/",partition_cols=['hash_key'])

## c.

In [134]:
df['src_airport_geohash'] = df.apply(
    lambda row: pygeohash.encode(row.src_airport_latitude, row.src_airport_longitude), axis=1
)

In [135]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key,src_airport_geohash
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6,szsrjjzd02b3
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9,v04pk3t5gbjj
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1,v04pk3t5gbjj
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3,v3gdxs17du83
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1,v3gdxs17du83


In [145]:
# df.count()

In [137]:
df.dropna(subset=['src_airport_iata', 'dst_airport_iata'], inplace=True)

In [146]:
#df.count()

In [147]:
#  locations = dict(
#         central=pygeohash.encode(41.1544433, -96.0422378),
#         ## TODO: add west and east
#         west=pygeohash.encode(45.5945645, -121.1786823),
#         east=pygeohash.encode(39.08344,  -77.6497145)
#   )

In [252]:
# print(locations)

In [253]:
# distance = list()
# for key, value in locations.items():
#     distance.append((pygeohash.geohash_haversine_distance(value,'szsrjjzd02b3'),key))
# #     .add(key,value)

# print(distance)

In [254]:
# distance.sort()
# print(distance)

In [255]:
# distance = (
#     ('central' , pygeohash.geohash_haversine_distance('9z7dnebnj8kb','szsrjjzd02b3')),
#     ('west' , pygeohash.geohash_haversine_distance('c21g6s0rs4c7','szsrjjzd02b3'))
# )
# type(distance)
# # distance[0][1]

In [245]:
def determine_location(src_airport_geohash):
    locations = dict(
        central=pygeohash.encode(41.1544433, -96.0422378),
        ## TODO: add west and east
        west=pygeohash.encode(45.5945645, -121.1786823),
        east=pygeohash.encode(39.08344,  -77.6497145)
      )
    #TODO: a list of centers and distances using the pygeohash.geohash_haversine_distance function
    distances = list()
    for key,value in locations.items():
        distances.append((pygeohash.geohash_haversine_distance(value,src_airport_geohash),key))
    
    distances.sort()
    return distances[0][1]
#     return distances

In [246]:
determine_location('szsrjjzd02b3')

'east'

In [247]:
df['location'] = df['src_airport_geohash'].apply(determine_location)

In [248]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key,src_airport_geohash,location
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6,szsrjjzd02b3,east
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9,v04pk3t5gbjj,east
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1,v04pk3t5gbjj,east
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3,v3gdxs17du83,west
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1,v3gdxs17du83,west


In [251]:
df['location'].value_counts()

east       39107
west       23033
central     4631
Name: location, dtype: int64

In [249]:
df.to_parquet('results/geo', partition_cols=['location'])

## d.

In [270]:
import numpy as np
keys = np.random.randint(low = 1, high =50, size =25)

In [271]:
keys.sort()
print(keys)


[ 1  3  4  5  6 14 14 15 19 20 22 29 31 31 32 37 38 38 41 42 43 45 45 46
 47]


In [277]:
import pandas as pd
pd.qcut(keys,5)


[(0.999, 12.4], (0.999, 12.4], (0.999, 12.4], (0.999, 12.4], (0.999, 12.4], ..., (42.2, 47.0], (42.2, 47.0], (42.2, 47.0], (42.2, 47.0], (42.2, 47.0]]
Length: 25
Categories (5, interval[float64]): [(0.999, 12.4] < (12.4, 21.2] < (21.2, 34.0] < (34.0, 42.2] < (42.2, 47.0]]

In [276]:
num_partitions = 7
num_in_first_bkt = len(keys)%num_partitions
num_in_other_bkt = (len(keys) - num_in_first_bkt)/num_partitions
print(num_in_first_bkt)

4


In [275]:
print(num_in_other_bkt)

3.0


In [ ]:
def balance_partitions(keys, num_partitions):
    orderd_keys = keys.sort()
    num_in_first_bkt = len(keys)%num_partitions
    num_in_other_bkt = (len(keys) - num_in_first_bkt)/num_partitions
    partitions = []
    
    return partitions